# gym中设置seed的作用范围
env.seed(): 如果设置了相同的seed，那么每次reset都是确定的，但每次reset未必是相同的，即保证的是环境初始化的一致性.

例如我们初始化环境env1，同时设置env1.seed(1)，然后reset()3次得到顺序的3个初始状态S1=[s0,s1,s2]。假如再初始化环境env2，也设置env2.seed(1)，那么reset()3次会得到与相同的3个初始状态S2=S1=[s0,s1,s2].

所以，在种子相同的情况下，理论上相同的确定性策略在环境中产生的轨迹也应该是相同的。但是做实验发现设置了相同seed的环境居然产生了不一样的轨迹。

测试代码如下，看到第一次reset后， env1和env2因为seed相同，所以s1==s2，没问题，并且s1 != s3. 符合预期。但是在env.action_space.sample()中发现 a1 != a2，说明 action_space.sample()不受env.seed()控制。

action_space是需要单独设置seed的。需要使用env.action_space.seed()保证

In [6]:
import gym
import numpy as np
env1 = gym.make('CartPole-v1')
env2 = gym.make('CartPole-v1')
env3 = gym.make('CartPole-v1')

env1.seed(1)
env2.seed(1)
env3.seed(2)

s1 = env1.reset() # [ 0.03073904  0.00145001 -0.03088818 -0.03131252]
s2 = env2.reset() # [ 0.03073904  0.00145001 -0.03088818 -0.03131252]
s3 = env3.reset() # [0.03468829 0.01500225 0.01230312 0.01825219]
print('s1={},\n s2={},\n s3={}'.format(s1,s2,s3))

a1 = env1.action_space.sample() # 0
a2 = env2.action_space.sample() # 1
a3 = env3.action_space.sample() # 0
print('a1={},a2={},a3={}'.format(a1,a2,a3))

next_obs1, _, _, _ = env1.step(a1) # [ 0.03076804 -0.19321569 -0.03151444  0.25146705]
next_obs2, _, _, _ = env2.step(a2) # [ 0.03076804  0.19700098 -0.03151444 -0.33357875]
next_obs3, _, _, _ = env3.step(a3) # [ 0.03498833 -0.18029396  0.01266816  0.31479136]

s4 = env1.reset() # [ 0.02725216 -0.04481721 -0.04304738  0.00151751]
s5 = env2.reset() # [ 0.02725216 -0.04481721 -0.04304738  0.00151751]
print('s4={},s5={}'.format(s4,
s5))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
s1=[ 0.03073904  0.00145001 -0.03088818 -0.03131252],
 s2=[ 0.03073904  0.00145001 -0.03088818 -0.03131252],
 s3=[0.03468829 0.01500225 0.01230312 0.01825219]
a1=0,a2=0,a3=0
s4=[ 0.02725216 -0.04481721 -0.04304738  0.00151751],s5=[ 0.02725216 -0.04481721 -0.04304738  0.00151751]


# torch.manual_seed()
设置CPU生成随机数的种子，方便下次复现实验结果。

```
torch.manual_seed(seed) → torch._C.Generator
```
seed (int) – CPU生成随机数的种子。取值范围为[-0x8000000000000000, 0xffffffffffffffff]，十进制是[-9223372036854775808, 18446744073709551615]，超出该范围将触发RuntimeError报错。

返回一个`torch.Generator`对象


In [8]:
# test.py
import torch
torch.manual_seed(0)
print(torch.rand(1)) # 返回一个张量，包含了从区间[0, 1)的均匀分布中抽取的一组随机数


tensor([0.4963])


# 注意
设置随机种子后，是每次运行test.py文件的输出结果都一样，而不是每次随机函数生成的结果一样：

In [9]:
# test.py
import torch
torch.manual_seed(0)
print(torch.rand(1))
print(torch.rand(1))


tensor([0.4963])
tensor([0.7682])


In [ ]:
# test.py
import torch
torch.manual_seed(0)
print(torch.rand(1))
torch.manual_seed(0)
print(torch.rand(1))


tensor([0.4963])
tensor([0.4963])


# torch.device()
常见用法
```
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
	device = torch.device("cuda")
else:
	device = torch.device("cpu")

```
这个device的用处是作为`Tensor`或者`Model`被分配到的位置。因此，在构建device对象后，紧跟的代码往往是：

```
data = data.to(device)
model = Model(...).to(device)
# 更一般地
torch.device('cuda', 0)
torch.device('cuda:0')

```

# 环境搭建
`env = gym.make(‘CartPole-v0’)` 创建环境

`env.reset()` 状态初始化

`env.render()`

`env.step()` 该函数在仿真器中扮演物理引擎的角色。其输入是动作a，输出是：下一步状态，立即回报，是否终止，调试项。




```
import gym
import time
env = gym.make('CartPole-v0')   #创造环境
observation = env.reset()       #初始化环境，observation为环境状态
count = 0
for t in range(100):
    action = env.action_space.sample()  #随机采样动作
    observation, reward, done, info = env.step(action)  #与环境交互，获得下一步的时刻
    if done:             
        break
    env.render()         #绘制场景
    count+=1
    time.sleep(0.2)      #每次等待0.2s
print(count)             #打印该次尝试的步数
```

In [1]:
import gym
import time
env = gym.make('CartPole-v0')   #创造环境
observation = env.reset()       #初始化环境，observation为环境状态
count = 0
for t in range(100):
    action = env.action_space.sample()  #随机采样动作
    observation, reward, done, info = env.step(action)  #与环境交互，获得下一步的时刻
    if done:             
        break
    env.render()         #绘制场景
    count+=1
    time.sleep(0.2)      #每次等待0.2s
print(count)             #打印该次尝试的步数


d:\env\Anaconda3\envs\py38\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
14


: 